# Homework 4
## Bangguo Wang

In [1]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Routine for decoding the CIFAR-10 binary file format."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import numpy as np
import time

# Process images of this size. Note that this differs from the original CIFAR
# image size of 32 x 32. If one alters this number, then the entire model
# architecture will change and any model would need to be retrained.
IMAGE_SIZE = 32

# Global constants describing the CIFAR-10 data set.
NUM_CLASSES = 10
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000


def read_cifar10(filename_queue):
    """Reads and parses examples from CIFAR10 data files.

    Recommendation: if you want N-way read parallelism, call this function
    N times.  This will give you N independent Readers reading different
    files & positions within those files, which will give better mixing of
    examples.

    Args:
        filename_queue: A queue of strings with the filenames to read from.

    Returns:
        An object representing a single example, with the following fields:
            height: number of rows in the result (32)
            width: number of columns in the result (32)
            depth: number of color channels in the result (3)
            key: a scalar string Tensor describing the filename & record number
                for this example.
            label: an int32 Tensor with the label in the range 0..9.
            uint8image: a [height, width, depth] uint8 Tensor with the image data
    """

    class CIFAR10Record(object):
        pass
    result = CIFAR10Record()

    # Dimensions of the images in the CIFAR-10 dataset.
    # See http://www.cs.toronto.edu/~kriz/cifar.html for a description of the
    # input format.
    label_bytes = 1  # 2 for CIFAR-100
    result.height = 32
    result.width = 32
    result.depth = 3
    image_bytes = result.height * result.width * result.depth
    # Every record consists of a label followed by the image, with a
    # fixed number of bytes for each.
    record_bytes = label_bytes + image_bytes

    # Read a record, getting filenames from the filename_queue.  No
    # header or footer in the CIFAR-10 format, so we leave header_bytes
    # and footer_bytes at their default of 0.
    reader = tf.FixedLengthRecordReader(record_bytes=record_bytes)
    result.key, value = reader.read(filename_queue)

    # Convert from a string to a vector of uint8 that is record_bytes long.
    record_bytes = tf.decode_raw(value, tf.uint8)

    # The first bytes represent the label, which we convert from uint8->int32.
    result.label = tf.cast(
            tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32)

    # The remaining bytes after the label represent the image, which we reshape
    # from [depth * height * width] to [depth, height, width].
    depth_major = tf.reshape(
            tf.strided_slice(record_bytes, [label_bytes],
                                             [label_bytes + image_bytes]),
            [result.depth, result.height, result.width])
    # Convert from [depth, height, width] to [height, width, depth].
    result.uint8image = tf.transpose(depth_major, [1, 2, 0])

    return result


def _generate_image_and_label_batch(image, label, min_queue_examples,
                                    batch_size, shuffle):
    """Construct a queued batch of images and labels.

    Args:
        image: 3-D Tensor of [height, width, 3] of type.float32.
        label: 1-D Tensor of type.int32
        min_queue_examples: int32, minimum number of samples to retain
            in the queue that provides of batches of examples.
        batch_size: Number of images per batch.
        shuffle: boolean indicating whether to use a shuffling queue.

    Returns:
        images: Images. 4D tensor of [batch_size, height, width, 3] size.
        labels: Labels. 1D tensor of [batch_size] size.
    """
    # Create a queue that shuffles the examples, and then
    # read 'batch_size' images + labels from the example queue.
    num_preprocess_threads = 8
    if shuffle:
        images, label_batch = tf.train.shuffle_batch(
                [image, label],
                batch_size=batch_size,
                num_threads=num_preprocess_threads,
                capacity=min_queue_examples + 3 * batch_size,
                min_after_dequeue=min_queue_examples)
    else:
        images, label_batch = tf.train.batch(
                [image, label],
                batch_size=batch_size,
                num_threads=num_preprocess_threads,
                capacity=min_queue_examples + 3 * batch_size)

    # Display the training images in the visualizer.
    tf.summary.image('images', images)

    return images, tf.reshape(label_batch, [batch_size])

def get_inputs(data_dir, batch_size, is_test=False):
    """Construct distorted input for CIFAR training using the Reader ops.

    Args:
        data_dir: Path to the CIFAR-10 data directory.
        batch_size: Number of images per batch.

    Returns:
        images: Images. 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
        labels: Labels. 1D tensor of [batch_size] size.
    """
    if not is_test:
        filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i)
                                 for i in xrange(1, 6)]
        num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN
    else:
        filenames = [os.path.join(data_dir, 'test_batch.bin')]
        num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_EVAL

    for f in filenames:
        if not tf.gfile.Exists(f):
            raise ValueError('Failed to find file: ' + f)

    # Create a queue that produces the filenames to read.
    filename_queue = tf.train.string_input_producer(filenames)

    # Read examples from files in the filename queue.
    read_input = read_cifar10(filename_queue)
    reshaped_image = tf.cast(read_input.uint8image, tf.float32)

    height = IMAGE_SIZE
    width = IMAGE_SIZE

    # Image processing for training the network. Note the many random
    # distortions applied to the image.

    # Randomly crop a [height, width] section of the image.


    # Because these operations are not commutative, consider randomizing
    # the order their operation.
    # NOTE: since per_image_standardization zeros the mean and makes
    # the stddev unit, this likely has no effect see tensorflow#1458.
    if not is_test:
        # Randomly flip the image horizontally.
        distorted_image = tf.random_crop(reshaped_image, [height, width, 3])
        distorted_image = tf.image.random_flip_left_right(distorted_image)
        # distorted_image = tf.image.random_brightness(distorted_image,max_delta=63)
        # distorted_image = tf.image.random_contrast(distorted_image,lower=0.2, upper=1.8)
        #distorted_image = distorted_image + tf.random_normal(shape=tf.shape(distorted_image), mean=0.0, stddev=0.2, dtype=tf.float32) 
    else:
        distorted_image = tf.image.resize_image_with_crop_or_pad(reshaped_image, height, width)

    # DON'T WANT TO DO THIS SINCE THE OUTPUT FROM THE GENERATOR IS SIGMOID
    # Subtract off the mean and divide by the variance of the pixels.
    # float_image = tf.image.per_image_standardization(distorted_image)
    
    #float_image = tf.multiply(distorted_image,1.0/255.0)
    float_image = tf.multiply(distorted_image,1.0/128.0)
    float_image = tf.add(float_image,-1.0)


    # Set the shapes of tensors.
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])

    # Ensure that the random shuffling has good mixing properties.
    min_fraction_of_examples_in_queue = 0.4
    min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN *
                                                     min_fraction_of_examples_in_queue)
    print ('Filling queue with %d CIFAR images before starting to train. '
                 'This will take a few minutes.' % min_queue_examples)

    # Generate a batch of images and labels by building up a queue of examples.
    return _generate_image_and_label_batch(float_image,read_input.label,min_queue_examples,batch_size,shuffle=True)

def train_inputs(batch_size):
    data_dir = 'cifar10_data/cifar-10-batches-bin/'
    images, labels = get_inputs(data_dir=data_dir,batch_size=batch_size)
    return images, labels

def test_inputs(batch_size):
    data_dir = 'cifar10_data/cifar-10-batches-bin/'
    images, labels = get_inputs(data_dir=data_dir,batch_size=batch_size, is_test=True)
    return images, labels

def plot(samples):
    fig = plt.figure(figsize=(8, 8))
    gs = gridspec.GridSpec(8, 8)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample)
    return fig

def plot_weights(samples):
    fig = plt.figure(figsize=(8, 8))
    gs = gridspec.GridSpec(8, 8)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        #ax.set_aspect('equal')
        plt.imshow(sample)
    return fig

def plot_classes(samples):
    fig = plt.figure(figsize=(10, 100))
    gs = gridspec.GridSpec(1, 10)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        #ax.set_aspect('equal')
        plt.imshow(sample)
    return fig

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

from model import discriminator,generator

batch_size = 64

with tf.device('/cpu:0'):
    images, labels = train_inputs(batch_size)
    images_test, labels_test = test_inputs(batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [2]:
with tf.variable_scope('placeholder'):
    X = tf.placeholder(tf.float32, [None, IMAGE_SIZE, IMAGE_SIZE, 3])
    y = tf.placeholder(name='label',dtype=tf.float32,shape=[None,10])
    keep_prob = tf.placeholder(tf.float32 ,shape=())
    is_train = tf.placeholder(tf.bool ,shape=())
    
with tf.variable_scope('GAN'):
    D, D_logits, flat_features = discriminator(X, 
        keep_prob=keep_prob, is_train=is_train, reuse=False)

with tf.variable_scope('D_loss'):
    label = tf.concat([y,tf.zeros([batch_size,1])],axis=1)
    d_loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
        logits=D_logits,labels=label))

with tf.variable_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(D[:,:-1],1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
with tf.name_scope('gradients'):
    grad_loss_over_X = tf.gradients(d_loss, X)[0]

    grad_features_over_X = tf.gradients(
        tf.reduce_mean(tf.diag_part(flat_features[0:64,0:64])),X)[0]
    grad_logit_over_X = tf.gradients(
        tf.reduce_mean(tf.diag_part(D_logits[0:10,0:10])),X)[0]

In [3]:
dvar = tf.global_variables()
saver = tf.train.Saver(dvar)

sess = tf.InteractiveSession()

init = tf.global_variables_initializer()
sess.run(init)

saver.restore(sess,tf.train.latest_checkpoint('GAN/discriminator/'))
#saver.restore(sess,tf.train.latest_checkpoint('discriminator_no_GAN/'))

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)
tf.train.start_queue_runners()

[]

In [4]:
X_batch = np.load('X_batch_1.npy')
labels_batch = np.load('labels_batch_1.npy')

In [5]:
#X_batch, labels_batch = sess.run([images_test, labels_test])

real_labels = labels_batch
alternate_labels = labels_batch + 1
alternate_labels[alternate_labels>=10]=0

y_batch_real = np.zeros((batch_size,10))
y_batch_real[np.arange(batch_size),real_labels] = 1

y_batch_alternate = np.zeros((batch_size,10))
y_batch_alternate[np.arange(batch_size),alternate_labels] = 1

In [10]:
np.save('X_batch_1', X_batch)
np.save('labels_batch_1',labels_batch)

In [6]:
X_batch

array([[[[ 0.84375  ,  0.8515625,  0.890625 ],
         [ 0.8125   ,  0.8203125,  0.859375 ],
         [ 0.8125   ,  0.8203125,  0.859375 ],
         ..., 
         [ 0.875    ,  0.8984375,  0.953125 ],
         [ 0.875    ,  0.8984375,  0.953125 ],
         [ 0.890625 ,  0.90625  ,  0.9453125]],

        [[ 0.859375 ,  0.8671875,  0.90625  ],
         [ 0.8203125,  0.828125 ,  0.8671875],
         [ 0.8203125,  0.828125 ,  0.8671875],
         ..., 
         [ 0.875    ,  0.8984375,  0.953125 ],
         [ 0.8828125,  0.8984375,  0.953125 ],
         [ 0.8984375,  0.90625  ,  0.9453125]],

        [[ 0.875    ,  0.8828125,  0.921875 ],
         [ 0.84375  ,  0.8515625,  0.890625 ],
         [ 0.8359375,  0.84375  ,  0.8828125],
         ..., 
         [ 0.8828125,  0.8984375,  0.9375   ],
         [ 0.8984375,  0.90625  ,  0.953125 ],
         [ 0.9140625,  0.921875 ,  0.9609375]],

        ..., 
        [[ 0.875    ,  0.875    ,  0.890625 ],
         [ 0.8515625,  0.8515625,  0.86718

### Perturb Real Images

In [6]:
gradient, pred, logit, correct = sess.run(
    [grad_loss_over_X,D,D_logits,correct_prediction], 
    feed_dict={X: X_batch, y: y_batch_real, keep_prob:1.0, is_train:False})

In [8]:
accuracy_1 = sum(correct)*1.0/len(correct)
print(correct)
print(accuracy_1)

[ True  True  True  True  True  True False  True  True  True  True False
  True False  True  True  True  True  True  True  True  True  True  True
  True False False  True  True  True  True False  True  True  True  True
 False False  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True]
0.84375


In [9]:
# calculate the gradient based on the alternate labels
gradient = sess.run(grad_loss_over_X, 
    feed_dict={X:X_batch, y: y_batch_alternate, keep_prob:1.0, is_train:False})

gradient_image = (gradient - np.min(gradient))/(np.max(gradient)-np.min(gradient))
fig = plot(gradient_image)
plt.savefig('gradient.png', bbox_inches='tight')
plt.close(fig)

In [10]:
# update the x 
gradient[gradient>0.0] = 1.0
gradient[gradient<0.0] = -1.0

X_batch_modified = X_batch - 10.0*0.007843137*gradient
X_batch_modified[X_batch_modified>1.0] = 1.0
X_batch_modified[X_batch_modified<-1.0] = -1.0

pred_alternate, logit_alternate, correct_alternate = sess.run(
    [D,D_logits,correct_prediction],
    feed_dict={X:X_batch_modified, y: y_batch_real, keep_prob:1.0, is_train:False})
  
X_batch += 1.0
X_batch /= 2.0
X_batch[X_batch>1.0] = 1.0
X_batch[X_batch<0.0] = 0.0
fig = plot(X_batch)
plt.savefig('X.png', bbox_inches='tight')
plt.close(fig)

X_batch_modified += 1.0
X_batch_modified /= 2.0
X_batch_modified[X_batch_modified>1.0] = 1.0
X_batch_modified[X_batch_modified<0.0] = 0.0
fig = plot(X_batch_modified)
plt.savefig('X_alternate.png', bbox_inches='tight')
plt.close(fig)

In [11]:
print(correct_alternate)
print(sum(correct_alternate)*1.0/len(correct_alternate))

[False False False False False False False False False False  True False
  True False False False  True  True  True False False False  True False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True False  True False False  True  True False
 False False False  True]
0.171875


### Perturb Image of Noise

In [24]:
X_batch = np.load('X_batch_2.npy')
labels_batch = np.load('labels_batch_2.npy')

In [25]:
#X_batch, labels_batch = sess.run([images_test, labels_test])
real_labels = labels_batch
y_batch_real = np.zeros((batch_size,10))
y_batch_real[np.arange(batch_size),real_labels] = 1

noise = np.random.normal(0.0,1.0, [64, 32, 32, 3])
noise = ((noise - np.min(noise))/(np.max(noise)-np.min(noise))*2.0) - 1.0

X_batch_modified = X_batch - 10.0*0.007843137*noise
X_batch_modified[X_batch_modified>1.0] = 1.0
X_batch_modified[X_batch_modified<-1.0] = -1.0

pred1, logit_1, correct1 = sess.run(
    [D,D_logits,correct_prediction],
    feed_dict={X:X_batch, y: y_batch_real, keep_prob:1.0, is_train:False})

pred_alternate1, logit_alternate1, correct_alternate1 = sess.run(
    [D,D_logits,correct_prediction],
    feed_dict={X:X_batch_modified, y: y_batch_real, keep_prob:1.0, is_train:False})



X_batch += 1.0
X_batch /= 2.0
X_batch[X_batch>1.0] = 1.0
X_batch[X_batch<0.0] = 0.0
fig = plot(X_batch)
plt.savefig('X_random.png', bbox_inches='tight')
plt.close(fig)

noise += 1.0
noise /= 2.0
noise[noise>1.0] = 1.0
noise[noise<0.0] = 0.0
fig = plot(noise)
plt.savefig('noise.png', bbox_inches='tight')
plt.close(fig)


X_batch_modified += 1.0
X_batch_modified /= 2.0
X_batch_modified[X_batch_modified>1.0] = 1.0
X_batch_modified[X_batch_modified<0.0] = 0.0
fig = plot(X_batch_modified)
plt.savefig('X_random_alternate.png', bbox_inches='tight')
plt.close(fig)

ValueError: operands could not be broadcast together with shapes (10,32,32,3) (64,32,32,3) 

In [23]:
np.save('X_batch_2', X_batch)
np.save('labels_batch_2',labels_batch)

In [21]:
print(correct1)
print(sum(correct1)*1.0/len(correct1))
print(correct_alternate1)
print(sum(correct_alternate1)*1.0/len(correct_alternate1))

[ True  True  True False  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True False  True  True
 False  True  True  True False  True  True  True  True  True  True  True
 False  True  True  True  True  True  True False  True  True  True  True
  True  True False False  True  True  True  True  True  True  True  True
  True  True  True  True]
0.859375
[ True  True  True False  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True False  True False
 False  True  True  True False  True  True  True  True  True  True  True
  True  True False  True  True  True  True False  True  True  True  True
  True  True False False  True  True  True  True  True  True  True False
  True  True  True  True]
0.828125


### High Activation Images for Output Layer

In [29]:
X_batch,_ = sess.run([images_test, labels_test])
X_batch = np.mean(X_batch,axis=0)[np.newaxis,:,:,:]
X_batch = np.repeat(X_batch,10,axis=0)

X_batch_modified = 1.0*X_batch

t1 = time.time()
for i in xrange(1000):
    gradient, logits = sess.run([grad_logit_over_X, D_logits],
        feed_dict={X:X_batch_modified, keep_prob:1.0, is_train:False})

    X_batch_modified = X_batch_modified + 0.4*gradient - 0.005*X_batch_modified

    X_batch_modified[X_batch_modified>1.0] = 1.0
    X_batch_modified[X_batch_modified<-1.0] = -1.0
    if i % 50 == 0:
        print(i, logits[np.arange(10),np.arange(10)])

X_batch_save = X_batch_modified*1.0
X_batch_save += 1.0
X_batch_save /= 2.0
X_batch_save[X_batch_save>1.0] = 1.0
X_batch_save[X_batch_save<0.0] = 0.0
fig = plot_classes(X_batch_save)
plt.savefig('classes2_GAN_1000.png', bbox_inches='tight')
plt.close(fig)

0 [-3.63054276 -4.60294628 -5.40586519 -1.77940273 -2.53270292 -2.43257427
 -3.45420408 -6.25994778 -1.16589367 -4.82567406]
50 [  4.4542551    8.05078125  11.15617657  11.17124081   6.32089281
   9.12386894   7.90320587   8.79802608   9.22894764   7.72657442]
100 [ 11.13281631  35.53216934  15.68049812  13.97629642   8.28242207
  11.59186172  10.18539715  11.73096943  11.99155903  11.79137611]
150 [ 11.62435436  81.28720093  18.8614521   15.74494457  10.43326378
  12.95098019  11.80966949  13.29580021  13.47536087  14.0026083 ]
200 [  15.56831264  118.57044983   21.52464485   18.08165932   11.13035011
   13.95387936   12.98787689   14.79876614   14.37774181   16.47390747]
250 [  18.72556686  144.58511353   23.6154213    19.47046471   11.90424156
   14.66835403   14.1131506    15.86469841   15.5986414    18.01822472]
300 [  21.56478119  161.02929688   25.47886086   20.81217194   11.91865826
   15.5177021    13.69612694   16.39470291   16.01737595   19.34455299]
350 [  20.75218773  172.

### High Activation Image for Intermediate Features

In [30]:
X_batch,_ = sess.run([images_test, labels_test])
X_batch = np.mean(X_batch,axis=0)[np.newaxis,:,:,:]
X_batch = np.repeat(X_batch,64,axis=0)

X_batch_modified = 1.0*X_batch

t1 = time.time()
for i in xrange(1000):
    gradient, features = sess.run([grad_features_over_X, flat_features],
        feed_dict={X:X_batch_modified, keep_prob:1.0, is_train:False})

    X_batch_modified = X_batch_modified + 2.0*gradient - 0.001*X_batch_modified

    X_batch_modified[X_batch_modified>1.0] = 1.0
    X_batch_modified[X_batch_modified<-1.0] = -1.0
    if i % 50 == 0:
        print(i)

X_batch_save = X_batch_modified*1.0
X_batch_save += 1.0
X_batch_save /= 2.0
X_batch_save[X_batch_save>1.0] = 1.0
X_batch_save[X_batch_save<0.0] = 0.0
fig = plot(X_batch_save)
plt.savefig('features_cool_GAN.png', bbox_inches='tight')
plt.close(fig)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
